![](imgs/kodolamaczlogo.png)

# Przetwarzanie Big Data z użyciem Apache Spark

Autor notebooka: Jakub Nowacki.


## Analiza MovieLens z użyciem Spark SQL

[MovieLense](https://movielens.org/) to strona rekomendująca filmy na podstawie analizy danych z [IMDB](http://www.imdb.com/). Dane ze strony udostępnione są pod [tym adresem](http://grouplens.org/datasets/movielens/). Dane można znaleźć w `data/ml-100k`. Dokładniejszy opis danych znajduje się w pliku `data/ml-100k/README`.

In [ ]:
import pyspark
import pyspark.sql.functions as func
import pyspark.sql.types as types
import matplotlib
%matplotlib inline 

spark = pyspark.sql.SparkSession.builder \
    .appName('MovieLens') \
    .getOrCreate()
sc = spark.sparkContext
    
#sc = pyspark.SparkContext(appName='MovieLens')
#sqlContext = pyspark.sql.SQLContext(sc)

Poniżej definiujemy kilka funkcji narzędziowych.

In [ ]:
def to_bool(value):
    '''
    Converts values (0, 1 (non-zero)) to boolean
    
    @param value: int value to convert
    '''
    v = int(value)
    return False if v == 0 else True

In [ ]:
def data_from_csv(line):
    '''
    Converts a line of data table from CSV to DataFrame Row
    
    @param line: line of data row 
    @returns: Row of parsed values
    '''
    c = line.split('\t')
    
    row = dict()
    row['userId'] = int(c[0])
    row['itemId'] = int(c[1])
    row['rating'] = int(c[2])
    row['timestamp'] = int(c[3]) # Timestamp Unix to long ale w Python 3 int to zarówno int jak long z Python 2.
    
    return pyspark.Row(**row)

In [ ]:
def item_from_csv(line):
    '''
    Converts a line of item table from CSV to DataFrame Row
    
    @param line: line of item row 
    @returns: Row of parsed values
    '''
    c = line.split('|')
    
    row = dict()
    row['movieId'] = int(c[0])
    row['movieTitle'] = str(c[1])
    row['releaseDate'] = str(c[2])
    row['videoReleaseDate'] = str(c[3])
    row['imdbUrl'] = str(c[4])
    row['unknown'] = to_bool(c[5])
    row['action'] = to_bool(c[6])
    row['adventure'] = to_bool(c[7])
    row['animation'] = to_bool(c[8])
    row['childrens'] = to_bool(c[9])
    row['comedy'] = to_bool(c[10])
    row['crime'] = to_bool(c[11])
    row['documentary'] = to_bool(c[12])
    row['drama'] = to_bool(c[13])
    row['fantasy'] = to_bool(c[14])
    row['filmNoir'] = to_bool(c[15])
    row['horror'] = to_bool(c[16])
    row['musical'] = to_bool(c[17])
    row['mystery'] = to_bool(c[18])
    row['romance'] = to_bool(c[19])
    row['sciFi'] = to_bool(c[20])
    row['thriller'] = to_bool(c[21])
    row['war'] = to_bool(c[22])
    row['western'] = to_bool(c[23])
    
    return pyspark.Row(**row)

In [ ]:
def user_from_csv(line):
    '''
    Converts a line of user table from CSV to DataFrame Row
    
    @param line: line of user row 
    @returns: Row of parsed values
    '''
    c = line.split('|')
    
    row = dict()
    row['userId'] = int(c[0])
    row['age'] = str(c[1])
    row['gender'] = str(c[2])
    row['occupation'] = str(c[3])
    row['zipCode'] = str(c[4])
        
    return pyspark.Row(**row)

Wczytujemy dane do DataFrame.

In [ ]:
data_rdd = sc.textFile('data/ml-100k/u.data').map(data_from_csv)
data = spark.createDataFrame(data_rdd)
data.printSchema()
data.show()

In [ ]:
item_rdd = sc.textFile('data/ml-100k/u.item').map(item_from_csv)
item = spark.createDataFrame(item_rdd)
item.printSchema()
item.show()

In [ ]:
user_rdd = sc.textFile('data/ml-100k/u.user').map(user_from_csv)
user = spark.createDataFrame(user_rdd)
user.printSchema()
user.show()

## Zadania

* Znajdź wszystkie filmy z 1983 roku; ile ich jest?
* Policz liczność zawodów użytkowników.
* Znajdź 20 filmów z najwyższymi ocenami.
* ★ Znajdź ulubiony film (najlepiej oceniony) 20 najczęściej oceniających użytkowników.
